In [0]:
import re 
import os
import json
from tqdm import tqdm_notebook
import time
import numpy as np
import scipy
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.decomposition import TruncatedSVD
import gensim

warnings.filterwarnings('ignore')
matplotlib.rcParams['figure.figsize'] = (8, 8)
sns.set_style('whitegrid')

# Для обработки текстов
!pip3 install pymorphy2
import pymorphy2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.matutils import sparse2full

from sklearn.neural_network import MLPClassifier


Загрузите данные

Создайте папку и положите в неё данные (avito_train.csv и avito_category.csv). Путь к папке назвать DIR_PATH

In [0]:
!wget -O prac4.zip https://www.dropbox.com/s/g6iurjn7azy7uco/%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B4%D0%BB%D1%8F%204%20%D0%BF%D1%80%D0%B0%D0%BA%D0%B0.zip?dl=0
!unzip prac4.zip && mv 'данные для 4 прака' prac4_data

--2020-01-02 16:54:17--  https://www.dropbox.com/s/g6iurjn7azy7uco/%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B4%D0%BB%D1%8F%204%20%D0%BF%D1%80%D0%B0%D0%BA%D0%B0.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/g6iurjn7azy7uco/%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B4%D0%BB%D1%8F%204%20%D0%BF%D1%80%D0%B0%D0%BA%D0%B0.zip [following]
--2020-01-02 16:54:17--  https://www.dropbox.com/s/raw/g6iurjn7azy7uco/%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B4%D0%BB%D1%8F%204%20%D0%BF%D1%80%D0%B0%D0%BA%D0%B0.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc44b8a89b8f584ccbe4a89c8e56.dl.dropboxusercontent.com/cd/0/inline/AvZEedcrs4GlG9uNlm98O5qEI0cYiOxzuIad36XOT6er9-ssvV8cLbw2kZ5Fy5zdugk456GJipYsD1EQQB4d21I6RU9nvnFQMc8mz

In [0]:
DIR_PATH = 'prac4_data'

dat = pd.read_csv(os.path.join(DIR_PATH, 'avito.csv'))
cat_descr = pd.read_csv(os.path.join(DIR_PATH, 'avito_category.csv'))
dat['category_name'] = dat.category_id.map(dict(zip(cat_descr.category_id, cat_descr.name)))

In [0]:
dat.head()

,item_id,title,description,price,category_id,category_name
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19,Для дома и дачи|Мебель и интерьер|Предметы инт...
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22,Для дома и дачи|Мебель и интерьер|Столы и стулья
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37,Личные вещи|Красота и здоровье|Приборы и аксес...
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43,Хобби и отдых|Книги и журналы|Книги
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1,Бытовая электроника|Ноутбуки


### Задание 1
### Подготовка данных

[1.1] (0.5 балла)

Разбейте таргет на иерархические классы.

In [0]:
dat['tmp'] = dat['category_name'].apply(lambda x: x.split('|'))
dat['category_1'] = dat['tmp'].apply(lambda x: x[0] if len(x) > 0 else '')
dat['category_2'] = dat['tmp'].apply(lambda x: x[1] if len(x) > 1 else '')
dat['category_3'] = dat['tmp'].apply(lambda x: x[2] if len(x) > 2 else '')
dat['category_4'] = dat['tmp'].apply(lambda x: x[3] if len(x) > 3 else '')

dat['last_cat'] = dat['tmp'].apply(lambda x: x[-1])
del dat['tmp']

In [0]:
print(dat['category_1'].unique(), '\n')
print(dat['category_2'].unique(), '\n')
print(dat['category_3'].unique(), '\n')
print(dat['category_4'].unique(), '\n')

['Для дома и дачи' 'Личные вещи' 'Хобби и отдых' 'Бытовая электроника'] 

['Мебель и интерьер' 'Красота и здоровье' 'Книги и журналы' 'Ноутбуки'
 'Музыкальные инструменты' 'Товары для детей и игрушки' 'Часы и украшения'
 'Билеты и путешествия' 'Одежда, обувь, аксессуары' 'Спорт и отдых'
 'Растения' 'Ремонт и строительство' 'Товары для компьютера'
 'Аудио и видео' 'Игры, приставки и программы' 'Телефоны'
 'Посуда и товары для кухни' 'Охота и рыбалка' 'Велосипеды'
 'Коллекционирование' 'Настольные компьютеры' 'Бытовая техника'
 'Планшеты и электронные книги'] 

['Предметы интерьера, искусство' 'Столы и стулья' 'Приборы и аксессуары'
 'Книги' '' 'Гитары и другие струнные' 'Товары для кормления' 'Часы'
 'Путешествия' 'Женская одежда' 'Игрушки' 'Детская мебель'
 'Фитнес и тренажёры' 'Освещение' 'Инструменты' 'Детские коляски'
 'Учебная литература' 'Шкафы и комоды' 'Сетевое оборудование'
 'Телевизоры и проекторы' 'Игровые приставки' 'Samsung' 'Посуда'
 'Мониторы' 'Игры для приставок' 'Sony' 

Загрузите список стоп-слов https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.json

In [0]:
stop_words = np.ravel(pd.read_json('https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.json'))

[1.2] (1 балл)

Лемматизируйте категории (таргеты)

In [0]:
morph = pymorphy2.MorphAnalyzer()
dat['category_name_processed'] =  dat['category_name'].apply(lambda x: morph.parse(x)[0].normal_form)

[1.3] (2 балла)

Лемматизируйте заголовки и описания объявлений

In [0]:
dat['title_processed'] = dat['title'].apply(lambda x: morph.parse(x)[0].normal_form)
dat['description_processed'] = dat['description'].apply(lambda x: morph.parse(x)[0].normal_form)
dat['title&description'] = dat['title_processed'] + ' ' + dat['description_processed']

[1.4] (0.5 балла)

Разбейте данные на обучение, валидацию и отложенный тест в пропорции 60/30/10. Постройте распределений категорий в каждом из разбиений для того, чтобы убедиться, что распределения совпадают

In [0]:
np.random.seed(10) # Не меняем random seed!
tr_indexes, val_indexes, te_indexes = np.split(
    np.random.permutation(np.arange(489517)), [int(489517*0.6), int(489517*0.9)]
)

vectorizer = TfidfVectorizer(stop_words = list(stop_words))
X = vectorizer.fit_transform(dat['title&description'])

tr = X[tr_indexes]; tr_target = dat['category_id'][tr_indexes]
val = X[val_indexes]; val_target = dat['category_id'][val_indexes]
test = X[te_indexes]; test_target = dat['category_id'][te_indexes]

tr, val, test = 
tr_target, val_target, test_target = dat['category_id'][tr_indexes], dat['category_id'][val_indexes], dat['category_id'][te_indexes]


SyntaxError: ignored

In [0]:
dat['category_id'][tr_indexes].hist()
dat['category_id'][val_indexes].hist()
dat['category_id'][te_indexes].hist()

### Задание 2
### Бейзлайн

[2.1] (5 баллов)

Обучите простую модель для того, чтобы в дальнейшем с ней сравниваться. Точность (accuracy) данной бейзлан модели должна быть > 0.6. 

В качестве такой модели можно взять Tf-Idf преобразования текстов + (logreg/random forest/boosting/...). Можно так же попробовать в качестве признаков взять word2vec вектора слов.

In [0]:
tr_target, val_target, test_target = dat['category_1'][tr_indexes], dat['category_1'][val_indexes], dat['category_1'][te_indexes]

lr = LogisticRegression()
lr.fit(tr, tr_target) 

print(accuracy_score(lr.predict(val), val_target))
print(accuracy_score(lr.predict(test), test_target))

cat_1 = lr.predict(X)

In [0]:
tr_target, val_target, test_target = dat['category_2'][tr_indexes], dat['category_2'][val_indexes], dat['category_2'][te_indexes]

lr = LogisticRegression()
lr.fit(tr, tr_target) 

print(accuracy_score(lr.predict(val), val_target))
print(accuracy_score(lr.predict(test), test_target))

cat_2 = lr.predict(X)

In [0]:
lr = LogisticRegression()
lr.fit(tr, tr_target)
print(accuracy_score(lr.predict(val), val_target))


[2.2] (2 балла)

Посчитайте качество вашей бейзлайн модели на отложенном тесте (te_indexe)

In [0]:
accuracy_score(lr.predict(test), test_target)

In [0]:
dat.to_csv('processed_prac4.csv', index = False)

### Задание 3

### Нейросетки

Настало время для действительно крутых моделей!

[3.1] (26 баллов) [Точность на тесте 0.6 - 0 баллов, точность на тесте 0.86 и более - 26 баллов]

Обучите нейросетку, которая бы решала данную задачу с точностью > 0.86. Вы неограничены в выборе архитектуры нейросети. Можно начать с Tf-Idf + полносвязная сеть. Экспериментируйте!

In [0]:
# READ DATA AND VECTORIZE

dat = pd.read_csv('processed_prac4.csv')
stop_words = np.ravel(pd.read_json('https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.json'))

X = TfidfVectorizer(stop_words = set(stop_words)).fit_transform(dat['title&description'])

In [0]:
#SPLIT 

np.random.seed(10) # Не меняем random seed!
tr_indexes, val_indexes, te_indexes = np.split(
    np.random.permutation(np.arange(489517)), [int(489517*0.6), int(489517*0.9)]
)
tr_target, val_target, test_target = dat['category_id'].values[tr_indexes], dat['category_id'].values[val_indexes], dat['category_id'].values[te_indexes]

In [0]:
# SKLEARN MLP

mlp = MLPClassifier()
mlp.fit(X[tr_indexes], tr_target)

accuracy_score(mlp.predict(X[val_indexes]), val_target)

0.8704232065642982

In [0]:
#pretrained Russian embeddings
!wget -O fasttext.tgz http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
!tar xzf fasttext.tgz

In [0]:
# TFIDF-weighted wor2vec

model = gensim.models.KeyedVectors.load('araneum_none_fasttextcbow_300_5_2018.model')

vectorizer = TfidfVectorizer(stop_words = list(stop_words), max_features = 12000)
Y = vectorizer.fit_transform(dat['title&description'])

emb_vecs = np.vstack([ model[word] if word in model.vocab else np.zeros(300) for word in vectorizer.get_feature_names()])
Y = np.vstack([np.dot(y.toarray(), emb_vecs) for y in Y])

In [0]:
# SVD

features = 800   # больше 800 не получается: недостаточно RAM. Желательно 4000-5000 фичей
X = TruncatedSVD(features).fit_transform(X)

In [0]:
price = dat['price'].values/dat['price'].values.max()
X = np.concatenate([X, Y, np.expand_dims(price, axis = 1)], axis = 1)

In [0]:
# TENSORFLOW

n_cats = len(dat['category_id'].unique())
model = tf.keras.Sequential([
    tf.keras.layers.Input(features + 301),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='sigmoid'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(dat['category_id'].unique()), activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(tr, tr_target, epochs=5)

In [0]:
model.evaluate(X[val_indexes], val_target)
model.evaluate(X[test_indexes], test_target)

In [0]:
# TORCH

hidden_layers = int((features + 1) * n_cats**(1/2))
model = nn.Sequential(
    nn.Linear(features, hidden_layers),
    nn.Dropout(0.1),
    nn.Linear(hidden_layers, n_cats),
    nn.Sigmoid()
)

optimizer = optim.Adamax(model.parameters(), lr=0.009)
epochs = 3
batch_size = 1024

def iterate_minibatches(X, y, batchsize):
    indexes = np.random.permutation(np.arange(len(X)))
    for start in range(0, len(indexes), batchsize):
        indexes = indexes[start: start + batchsize]
        yield X[indexes], y[indexes]
        
def compute_loss_finction(X_batch, y_batch):
    X_batch = torch.autograd.Variable(torch.FloatTensor(X_batch))
    y_batch = torch.autograd.Variable(torch.LongTensor(y_batch))
    logits = model(X_batch)
    return nn.functional.cross_entropy(logits, y_batch).mean()

for e in range(epochs):
    running_loss = 0
    for X_batch, y_batch in iterate_minibatches(X[train_indexes], tr_target, batch_size):
        loss = compute_loss_function(X_batch, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [0]:
predictions = np.argmax(model(torch.autograd.Variable(torch.FloatTensor(X[test_indexes]))), axis=1)
print(f'Accuracy: {accuracy_score(test_target, preds)}')